# Elo Merchant Category Recommendation 🪙

---

# Configuration ⚙️

In [3]:
# ---------Import----------
import gc
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb, xgboost as xgb

from scipy.stats import skew
from scipy.special import boxcox1p

from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge, BayesianRidge, LassoLarsIC

In [4]:
# ---------Set----------
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 100)        # Set the width to 1000 characters
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', '{:.4f}'.format)

In [5]:
def reduce_mem_usage(df, use_float16=False):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            # skip datetime type or categorical type
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            #df[col] = df[col].astype('category')
            pass

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

---

# Loading 📁 

In [6]:
# Loading
df_train = pd.read_csv('/Users/fuhan/Desktop/Kaggle/Elo Merchant Category Recommendation/dataset/train.csv')
df_test = pd.read_csv('/Users/fuhan/Desktop/Kaggle/Elo Merchant Category Recommendation/dataset/test.csv')
df_hist = pd.read_csv('/Users/fuhan/Desktop/Kaggle/Elo Merchant Category Recommendation/dataset/historical_transactions.csv',
                      parse_dates=['purchase_date'])
df_new = pd.read_csv('/Users/fuhan/Desktop/Kaggle/Elo Merchant Category Recommendation/dataset/new_merchant_transactions.csv',
                      parse_dates=['purchase_date'])

train_size = df_train.shape[0]

In [7]:
# Preliminary Missing Dealing
df_new['category_2'] = df_new['category_2'].fillna(0)
df_new['merchant_id'] = df_new['merchant_id'].fillna('No')
df_new['category_3'] = df_new['category_3'].fillna('N')

df_hist['category_2'] = df_hist['category_2'].fillna(0)
df_hist['merchant_id'] = df_hist['merchant_id'].fillna('No')
df_hist['category_3'] = df_hist['category_3'].fillna('N')

df_test['first_active_month'] = df_test['first_active_month'].fillna(df_test['first_active_month'].mode()[0])

In [8]:
# Concatenation
df_trian_test = pd.concat([df_train,df_test], axis=0, ignore_index=True)
df_purchase = pd.concat([df_hist,df_new], axis=0, ignore_index=True)

In [9]:
# Merge
df_total = df_trian_test.merge(df_purchase, on='card_id', how='left')

# Reduce Memory Use
df_trian_test = reduce_mem_usage(df_trian_test)
df_total = reduce_mem_usage(df_total)

del df_train
del df_hist
del df_test
del df_purchase
gc.collect()

Memory usage of dataframe is 14.90 MB
Memory usage after optimization is: 7.14 MB
Decreased by 52.1%
Memory usage of dataframe is 4741.73 MB
Memory usage after optimization is: 2607.95 MB
Decreased by 45.0%


0

---

# Preprocess

### Feature Engineering

In [8]:
# Extract useful columns from datatime
df_total['purchase_year'] = df_total['purchase_date'].dt.year
df_total['purchase_weekofyear'] = df_total['purchase_date'].dt.weekofyear
df_total['purchase_month'] = df_total['purchase_date'].dt.month
df_total['purchase_dayofweek'] = df_total['purchase_date'].dt.dayofweek
df_total['purchase_weekend'] = (df_total['purchase_date'].dt.weekday >=5).astype(int)
df_total['purchase_year'] = df_total['purchase_date'].dt.year
df_total['purchase_year_and_week'] = df_total['purchase_weekofyear'].astype(str) + '_' + df_total['purchase_year'].astype(str)
df_total['purchase_year_and_month'] = df_total['purchase_month'].astype(str) + '_' + df_total['purchase_year'].astype(str)

# Feature Engineering Results Key:card_id
df_card_id_group = df_total.groupby('card_id')

# Make total feature  
df_total_purchase_amount = df_total.groupby('card_id').agg(total_purchase=('first_active_month','count')).reset_index()

# Time Based Analysis
df_time_analysis = df_card_id_group.agg(
        nunique_year = ('purchase_year','nunique'),
        nunique_weekofyear = ('purchase_weekofyear','nunique'),
        nunique_purchase_month = ('purchase_month','nunique'),
        mode_purchase_month = ('purchase_month',lambda x:x.mode()[0]),    
        nunique_purchase_dayofweek = ('purchase_dayofweek','nunique'),
        mode_purchase_dayofweek = ('purchase_dayofweek',lambda x:x.mode()[0]),
        purchase_is_weekend_sum = ('purchase_weekend',lambda x:(x==1).sum()),
        purchase_not_weekend_sum = ('purchase_weekend',lambda x:(x==0).sum()),
        nunique_purchase_yweek = ('purchase_year_and_week','nunique'),
        nunique_purchase_ymonth = ('purchase_year_and_month','nunique')
        )

df_time_analysis.reset_index(inplace=True)

# df_total_purchase_amount['purchase_bin'] = pd.qcut(df_total_purchase_amount['total_purchase'],q=10)
df_overall_features = df_card_id_group.agg(total_amount=('purchase_amount','sum'),
                                             avg_amount=('purchase_amount','mean'),
                                             var_amount=('purchase_amount','var'),
                                             std_amount=('purchase_amount','std'),
                                             max_amount=('purchase_amount','max'),
                                             min_amount=('purchase_amount','min')
                                            ).reset_index()

df_overall_features['auto_binned'] =  pd.qcut(df_overall_features['total_amount'], q=10)
df_overall_features['auto_binned'] = df_overall_features['auto_binned'].cat.codes


# Big-Small Clientile

bottom_10pct_threshold = df_total['purchase_amount'].quantile(0.1)
bottom_1pct_threshold = df_total['purchase_amount'].quantile(0.01)

top_1pct_threshold = df_total['purchase_amount'].quantile(0.9)
top_01pct_threshold = df_total['purchase_amount'].quantile(0.99)
top_001pct_threshold = df_total['purchase_amount'].quantile(0.999)
top_0001pct_threshold = df_total['purchase_amount'].quantile(0.9999)

# Big Customer
df_card_id_approved_group = df_total[df_total['authorized_flag']=='Y'].groupby('card_id')

df_big_customer = df_card_id_approved_group.agg(
    big_1_pur = ('purchase_amount',lambda x:(x>top_1pct_threshold).sum()),
    big_01_pur = ('purchase_amount',lambda x:(x>top_01pct_threshold).sum()),
    big_001_pur = ('purchase_amount',lambda x:(x>top_001pct_threshold).sum()),
    big_0001_pur = ('purchase_amount',lambda x:(x>top_0001pct_threshold).sum())
)
df_big_customer.reset_index(inplace=True)

# Small pur
df_small_pur = df_card_id_approved_group.agg(
    small_pur = ('purchase_amount',lambda x:(x<bottom_10pct_threshold).sum()),
    tiny_pur = ('purchase_amount',lambda x:(x<bottom_1pct_threshold).sum())
)
df_small_pur.reset_index(inplace=True)

df_approved_time = df_card_id_group.agg(approved_time=('authorized_flag',lambda x:(x=='Y').sum())).reset_index()
df_app_small_pur = df_approved_time.merge(df_small_pur, on='card_id', how='left')
df_app_small_pur['small_rate'] = df_app_small_pur['small_pur']/df_app_small_pur['approved_time']
df_app_small_pur['tiny_rate'] = df_app_small_pur['tiny_pur']/df_app_small_pur['approved_time']
df_app_small_pur.drop('approved_time',axis=1, inplace=True)


df_merge_all = df_trian_test.merge(df_total_purchase_amount, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_overall_features, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_time_analysis, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_big_customer, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_app_small_pur, on='card_id', how='left')

del df_time_analysis
del df_approved_time
del df_card_id_approved_group
del df_overall_features
gc.collect()

18

In [9]:
# --------------------------------COMMON--------------------------------
# Make unique feature
df_unique_cities = df_total.groupby('card_id').agg(unique_cities=('city_id','nunique')).reset_index()
df_unique_states = df_total.groupby('card_id').agg(unique_states=('state_id','nunique')).reset_index()

df_unique_merchants = df_total.groupby('card_id').agg(unique_merchants=('merchant_id', 'nunique')).reset_index()
# df_unique_merchants['merchant_bin'] = pd.qcut(df_unique_merchants['unique_merchants'],q=10).reset_index()

df_uni_tran_month = df_total.groupby('card_id').agg(unique_states=('month_lag','nunique')).reset_index()

# Most active city
df_active_city = df_total.groupby('card_id')['city_id'].agg(lambda x: x.mode().iloc[0]).reset_index().rename(columns={'city_id': 'active_city_id'})

# Usage_months  till 2018-02
df_act_mon = df_trian_test[['card_id','first_active_month']].copy()
df_act_mon['first_active_month'] = pd.to_datetime(df_act_mon['first_active_month'])
reference_date = pd.to_datetime('2018-02')
df_act_mon['active_month'] = (reference_date.year - df_act_mon['first_active_month'].dt.year) * 12 + (reference_date.month - df_act_mon['first_active_month'].dt.month)
df_act_mon.drop('first_active_month',axis=1,inplace=True)


df_merge_all = df_merge_all.merge(df_unique_cities, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_unique_states, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_unique_merchants, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_uni_tran_month, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_active_city, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_act_mon, on='card_id', how='left')


del df_unique_cities
del df_unique_states
del df_unique_merchants
del df_uni_tran_month
del df_active_city
del df_act_mon
gc.collect()

21

In [10]:
# -----------------------------INSTALLMENT-----------------------------

# installment flaw times

df_installment_flaw=df_card_id_group.agg(
    flaw_istmt_times_999=('installments',lambda x:(x==999).sum()),
    flaw_istmt_times_minus1=('installments',lambda x:(x==-1).sum())
).reset_index()

df_istmt_without_flaw = df_total[(df_total['installments'] != 999) & (df_total['installments'] > 0)]

# total & avg installments(without flaws)
df_total_inst = df_istmt_without_flaw.groupby('card_id').agg(sum_instmts = ('installments','sum')).reset_index()
df_avg_inst = df_istmt_without_flaw.groupby('card_id').agg(avg_instmts = ('installments','mean')).reset_index()

del df_istmt_without_flaw
gc.collect()

# Payment approved times
df_approved_time = df_card_id_group.agg(approved_time=('authorized_flag',lambda x:(x=='Y').sum())).reset_index()

# Payment rejected times
df_rejected_time = df_card_id_group.agg(rejected_time=('authorized_flag',lambda x:(x=='N').sum())).reset_index()

# Payment approved ratio
df_app_total = df_approved_time.merge(df_total_purchase_amount,on='card_id',how='left')
df_app_total['app_ratio'] = df_app_total['approved_time'] / df_app_total['total_purchase']

df_app_ratio = df_app_total[['card_id','app_ratio']]

del df_app_total
gc.collect()

# Payment rejected ratio 
df_rej_total = df_rejected_time.merge(df_total_purchase_amount,on='card_id',how='left')
df_rej_total['rej_ratio'] = df_rej_total['rejected_time'] / df_rej_total['total_purchase']

df_rej_ratio = df_rej_total[['card_id','rej_ratio']]

del df_rej_total
gc.collect()

df_merge_all = df_merge_all.merge(df_installment_flaw, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_total_inst, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_avg_inst, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_approved_time, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_rejected_time, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_app_ratio, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_rej_ratio, on='card_id', how='left')

del df_installment_flaw
del df_total_inst
del df_avg_inst
del df_approved_time
del df_rejected_time
del df_app_ratio
del df_rej_ratio

gc.collect()

# installment total times
df_inst_time = df_card_id_group.agg(inst_time=('installments', lambda x:((x>0) & (x!=999)).sum()))
df_inst_time = df_inst_time.reset_index()
df_merge_all = df_merge_all.merge(df_inst_time, on='card_id', how='left')
del df_inst_time
gc.collect()


# installment rejected times
df_inst_rej = df_total[(df_total['installments']!=0) & (df_total['authorized_flag']=='N')]                                  
df_inst_rej_time = df_inst_rej.groupby('card_id').agg(inst_rej_time = ('first_active_month','count'))
df_inst_rej_time = df_inst_rej_time.reset_index()

df_merge_all = df_merge_all.merge(df_inst_rej_time, on='card_id', how='left')
del df_inst_rej
del df_inst_rej_time
gc.collect()


# installment approved times
df_inst_app = df_total[(df_total['installments']!=0) & (df_total['authorized_flag']=='Y')]                                  
df_inst_app_time = df_inst_app.groupby('card_id').agg(inst_app_time = ('first_active_month','count'))
df_inst_app_time = df_inst_app_time.reset_index()

df_merge_all = df_merge_all.merge(df_inst_app_time, on='card_id', how='left')
del df_inst_app
del df_inst_app_time
gc.collect()


# no installment & is installment & installment rate
df_is_no_inst = df_card_id_group.agg(no_inst_times=('installments', lambda x:(x==0).sum()) ,
                     is_inst_times=('installments', lambda x:((x>0)&(x!=999)).sum()),
                     total_purchase=('installments', 'count'))
df_is_no_inst['is_insta_rate'] = df_is_no_inst['is_inst_times']/(df_is_no_inst['total_purchase']+1)
df_is_no_inst['no_insta_rate'] = df_is_no_inst['no_inst_times']/(df_is_no_inst['total_purchase']+1)
df_is_no_inst.drop('total_purchase', axis=1)
df_is_no_inst = df_is_no_inst.reset_index()

df_merge_all = df_merge_all.merge(df_is_no_inst, on='card_id', how='left')
del df_is_no_inst
gc.collect()

0

In [11]:
# ---------------------TIME SERIES---------------------

# Monthly Aggregates: Group the transactions by month_lag and compute aggregate features
# 1. Total transaction amount
df_mon_trans_amount = df_card_id_group.agg(
    **{f'mon_lag{i}_trans_time': ('month_lag', lambda x, i=i: (x == -i).sum()) for i in range(1, 14)},
    **{f'mon_forward{i}_tans_time': ('month_lag', lambda x, i=i: (x == i).sum()) for i in range(0, 3)}
)
df_mon_trans_amount = df_mon_trans_amount.reset_index()

# 2. Average purchase value
df_mon_pur_amount = df_total.groupby(['card_id','month_lag']).agg(month_pur_sum=('purchase_amount','sum'),
                     month_pur_avg=('purchase_amount','mean'),
                     month_pur_var=('purchase_amount','var')).reset_index()

pivoted_df = df_mon_pur_amount.pivot(index='card_id', columns='month_lag', values=['month_pur_sum', 'month_pur_avg', 'month_pur_var'])
pivoted_df.columns = [f"{col[0]}_{int(col[1])}" for col in pivoted_df.columns]
df_mon_pur_amount = pivoted_df.reset_index()

df_merge_all = df_merge_all.merge(df_mon_trans_amount, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_mon_pur_amount, on='card_id', how='left')

del df_mon_pur_amount
del df_mon_trans_amount
gc.collect()

# Rolling Windows: Create rolling aggregates over different time windows to capture recent trends and behaviors.
# (e.g., 3-month or 6-month rolling averages)
# Trend Indicator: Calculate the difference in transaction amounts or 
#                  counts between consecutive months (month_lag). 
#                  This will help identify whether a customer's spending behavior is
#                  increasing or decreasing over time.

df_rolling_pre = (
    df_total[['card_id', 'month_lag', 'installments', 'purchase_amount']]
    .groupby(['card_id', 'month_lag'])
    .agg(
        mon_avg=('purchase_amount', 'mean')
    )
    .reset_index()
)

df_rolling_pre.set_index('month_lag', inplace=True)

df_rolling_pre['rolling_3m'] = (
    df_rolling_pre.groupby('card_id')['mon_avg']
    .rolling(window=3, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)
df_rolling_pre['rolling_6m'] = (
    df_rolling_pre.groupby('card_id')['mon_avg']
    .rolling(window=6, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)
df_rolling_pre['rolling_12m'] = (
    df_rolling_pre.groupby('card_id')['mon_avg']
    .rolling(window=12, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

df_rolling_pre.reset_index(inplace=True)
df_rolling_pre.set_index('card_id',inplace=True)

df_0_b1_roldif = df_rolling_pre[df_rolling_pre['month_lag']==0][['rolling_3m']]-df_rolling_pre[df_rolling_pre['month_lag']==-1][['rolling_3m']]
df_0_b1_roldif.rename(columns={'rolling_3m': 'diff_0_-1'}, inplace=True)
df_0_b1_roldif.reset_index(inplace=True)

df_b1_b2_roldif = df_rolling_pre[df_rolling_pre['month_lag']==-1][['rolling_3m']]-df_rolling_pre[df_rolling_pre['month_lag']==-2][['rolling_3m']]
df_b1_b2_roldif.rename(columns={'rolling_3m': 'diff_-1_-2'}, inplace=True)
df_b1_b2_roldif.reset_index(inplace=True)

df_b2_b3_roldif = df_rolling_pre[df_rolling_pre['month_lag']==-2][['rolling_3m']]-df_rolling_pre[df_rolling_pre['month_lag']==-3][['rolling_3m']]
df_b2_b3_roldif.rename(columns={'rolling_3m': 'diff_-2_-3'}, inplace=True)
df_b2_b3_roldif.reset_index(inplace=True)

df_b3_b4_roldif = df_rolling_pre[df_rolling_pre['month_lag']==-3][['rolling_3m']]-df_rolling_pre[df_rolling_pre['month_lag']==-4][['rolling_3m']]
df_b3_b4_roldif.rename(columns={'rolling_3m': 'diff_-3_-4'}, inplace=True)
df_b3_b4_roldif.reset_index(inplace=True)

df_merge_all = df_merge_all.merge(df_0_b1_roldif, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_b1_b2_roldif, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_b2_b3_roldif, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_b3_b4_roldif, on='card_id', how='left')


del df_0_b1_roldif
del df_b1_b2_roldif
del df_b2_b3_roldif
del df_b3_b4_roldif
gc.collect()

df_b4_b5_roldif = df_rolling_pre[df_rolling_pre['month_lag']==-4][['rolling_3m']]-df_rolling_pre[df_rolling_pre['month_lag']==-5][['rolling_3m']]
df_b4_b5_roldif.rename(columns={'rolling_3m': 'diff_-4_-5'}, inplace=True)
df_b4_b5_roldif.reset_index(inplace=True)

df_b5_b6_roldif = df_rolling_pre[df_rolling_pre['month_lag']==-5][['rolling_3m']]-df_rolling_pre[df_rolling_pre['month_lag']==-6][['rolling_3m']]
df_b5_b6_roldif.rename(columns={'rolling_3m': 'diff_-5_-6'}, inplace=True)
df_b5_b6_roldif.reset_index(inplace=True)

df_b6_b7_roldif = df_rolling_pre[df_rolling_pre['month_lag']==-6][['rolling_3m']]-df_rolling_pre[df_rolling_pre['month_lag']==-7][['rolling_3m']]
df_b6_b7_roldif.rename(columns={'rolling_3m': 'diff_-6_-7'}, inplace=True)
df_b6_b7_roldif.reset_index(inplace=True)

df_b7_b8_roldif = df_rolling_pre[df_rolling_pre['month_lag']==-7][['rolling_3m']]-df_rolling_pre[df_rolling_pre['month_lag']==-8][['rolling_3m']]
df_b7_b8_roldif.rename(columns={'rolling_3m': 'diff_-7_-8'}, inplace=True)
df_b7_b8_roldif.reset_index(inplace=True)

df_f1_0_roldif = df_rolling_pre[df_rolling_pre['month_lag']==1][['rolling_3m']]-df_rolling_pre[df_rolling_pre['month_lag']==0][['rolling_3m']]
df_f1_0_roldif.rename(columns={'rolling_3m': 'diff_1_0'}, inplace=True)
df_f1_0_roldif.reset_index(inplace=True)

df_f2_f1_roldif = df_rolling_pre[df_rolling_pre['month_lag']==2][['rolling_3m']]-df_rolling_pre[df_rolling_pre['month_lag']==1][['rolling_3m']]
df_f2_f1_roldif.rename(columns={'rolling_3m': 'diff_2_1'}, inplace=True)
df_f2_f1_roldif.reset_index(inplace=True)

df_merge_all = df_merge_all.merge(df_b4_b5_roldif, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_b5_b6_roldif, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_b6_b7_roldif, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_b7_b8_roldif, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_f1_0_roldif, on='card_id', how='left')
df_merge_all = df_merge_all.merge(df_f2_f1_roldif, on='card_id', how='left')


del df_b4_b5_roldif
del df_b5_b6_roldif
del df_b6_b7_roldif
del df_b7_b8_roldif
del df_f1_0_roldif
del df_f2_f1_roldif
del df_card_id_group
del df_total_purchase_amount
gc.collect()

0

In [12]:
df_merge_all = reduce_mem_usage(df_merge_all)

Memory usage of dataframe is 242.78 MB
Memory usage after optimization is: 125.74 MB
Decreased by 48.2%


In [13]:
df_merge_all

,first_active_month,card_id,feature_1,feature_2,feature_3,target,total_purchase_x,total_amount,avg_amount,var_amount,std_amount,max_amount,min_amount,auto_binned,nunique_year,nunique_weekofyear,nunique_purchase_month,mode_purchase_month,nunique_purchase_dayofweek,mode_purchase_dayofweek,purchase_is_weekend_sum,purchase_not_weekend_sum,nunique_purchase_yweek,nunique_purchase_ymonth,big_1_pur,big_01_pur,big_001_pur,big_0001_pur,small_pur,tiny_pur,small_rate,tiny_rate,unique_cities,unique_states_x,unique_merchants,unique_states_y,active_city_id,active_month,flaw_istmt_times_999,flaw_istmt_times_minus1,sum_instmts,avg_instmts,approved_time,rejected_time,app_ratio,rej_ratio,inst_time,inst_rej_time,inst_app_time,no_inst_times,is_inst_times,total_purchase_y,is_insta_rate,no_insta_rate,mon_lag1_trans_time,mon_lag2_trans_time,mon_lag3_trans_time,mon_lag4_trans_time,mon_lag5_trans_time,mon_lag6_trans_time,mon_lag7_trans_time,mon_lag8_trans_time,mon_lag9_trans_time,mon_lag10_trans_time,mon_lag11_trans_time,mon_lag12_trans_time,mon_lag13_trans_time,mon_forward0_tans_time,mon_forward1_tans_time,mon_forward2_tans_time,month_pur_sum_-13,month_pur_sum_-12,month_pur_sum_-11,month_pur_sum_-10,month_pur_sum_-9,month_pur_sum_-8,month_pur_sum_-7,month_pur_sum_-6,month_pur_sum_-5,month_pur_sum_-4,month_pur_sum_-3,month_pur_sum_-2,month_pur_sum_-1,month_pur_sum_0,month_pur_sum_1,month_pur_sum_2,month_pur_avg_-13,month_pur_avg_-12,month_pur_avg_-11,month_pur_avg_-10,month_pur_avg_-9,month_pur_avg_-8,month_pur_avg_-7,month_pur_avg_-6,month_pur_avg_-5,month_pur_avg_-4,month_pur_avg_-3,month_pur_avg_-2,month_pur_avg_-1,month_pur_avg_0,month_pur_avg_1,month_pur_avg_2,month_pur_var_-13,month_pur_var_-12,month_pur_var_-11,month_pur_var_-10,month_pur_var_-9,month_pur_var_-8,month_pur_var_-7,month_pur_var_-6,month_pur_var_-5,month_pur_var_-4,month_pur_var_-3,month_pur_var_-2,month_pur_var_-1,month_pur_var_0,month_pur_var_1,month_pur_var_2,diff_0_-1,diff_-1_-2,diff_-2_-3,diff_-3_-4,diff_-4_-5,diff_-5_-6,diff_-6_-7,diff_-7_-8,diff_1_0,diff_2_1
0,2017-06,C_ID_92a2005557,5,2,1,-0.8203,283,-179.2129,-0.6333,0.0431,0.2075,2.2584,-0.7394,0,2,42,11,12,7,5,96,187,42,11,14,1,0,0,9,0,0.0333,0.0000,9,3,118,11,69,8,0,0,4.0000,1.0000,270,13,0.9541,0.0459,4,4.0000,NaN,279,4,283,0.0141,0.9824,21,57,21,22,20,44,49,3,0,0,0,0,0,23,12,11,NaN,NaN,NaN,NaN,NaN,-2.0713,-32.8268,-30.9824,-12.7850,-14.4114,-13.5147,-30.2848,-13.5425,-15.5499,-6.6900,-6.5542,NaN,NaN,NaN,NaN,NaN,-0.6904,-0.6699,-0.7041,-0.6392,-0.6551,-0.6436,-0.5313,-0.6449,-0.6761,-0.5575,-0.5958,NaN,NaN,NaN,NaN,NaN,0.0004,0.0075,0.0012,0.0060,0.0072,0.0074,0.1699,0.0141,0.0043,0.0217,0.0159,-0.0108,0.0034,0.0360,0.0202,0.0050,0.0171,-0.0080,0.0102,-0.0087,0.0163
1,2017-01,C_ID_3d0044924f,4,1,0,0.3929,356,-214.3621,-0.6021,0.1460,0.3820,4.6303,-0.7424,0,2,50,12,1,7,5,132,224,58,15,30,1,0,0,35,0,0.1014,0.0000,9,3,148,15,69,13,0,2,551.0000,1.5565,345,11,0.9691,0.0309,354,11.0000,345.0000,0,354,356,0.9916,0.0000,47,16,26,24,14,49,34,15,18,13,22,21,0,51,3,3,NaN,-7.4025,-12.4741,-5.6077,-10.3627,-9.0552,-22.5433,-31.6385,-7.9409,-9.4059,-16.2081,-10.5958,-31.7136,-35.0582,-2.2047,-2.1511,NaN,-0.3525,-0.5670,-0.4314,-0.5757,-0.6037,-0.6630,-0.6457,-0.5672,-0.3919,-0.6234,-0.6622,-0.6748,-0.6874,-0.7349,-0.7170,NaN,1.3456,0.0428,0.2480,0.0403,0.0872,0.0165,0.0767,0.0814,0.3462,0.0087,0.0120,0.0281,0.0075,0.0000,0.0003,-0.0213,-0.0943,-0.0317,0.0074,0.0904,0.0122,-0.0233,-0.0772,-0.0242,-0.0141
2,2016-08,C_ID_d639edf6cd,2,2,0,0.6881,44,-29.8677,-0.6788,0.0075,0.0864,-0.1458,-0.7301,5,2,22,10,1,7,4,12,32,25,13,1,0,0,0,0,0,0.0000,0.0000,5,2,14,13,143,18,0,0,NaN,NaN,42,2,0.9545,0.0455,0,NaN,NaN,44,0,44,0.0000,0.9778,2,1,0,6,1,2,2,0,4,7,5,6,6,1,0,1,-4.1713,-4.1001,-3.5599,-4.8170,-2.7491,NaN,-1.3941,-1.3866,-0.7020,-4.0850,NaN,-0.7078,-0.8333,-0.6613,NaN,-0.7003,-0.6952,-0.6833,-0.7120,-0.6881,-0.6873,NaN,-0.6970,-0.6933,-0.7020,-0.6808,NaN,-0.7078,-0.4166,-0.6613,NaN,-0.7003,0.0010,0.0016,0.0006,0.0007,0.0007,NaN,0.0005,0.0002,

In [14]:
train_size

201917

In [15]:
df_merge_all.to_csv('after_engineering.csv',index=False)